In [1]:
"""
Alexander Brown, 3/22
"""

""" import necessary libraries """

import os
import soundfile as sf
import librosa
import random
import soundfile as sf
import numpy as np
import csv 

from pydub import AudioSegment


In [2]:
# user-specific variables
char_path = 'C:/Users/eholy/Desktop/SD/farfan_files/AFarfan/'
noisepool_path = 'C:/Users/eholy/Desktop/SD/voxshattered/'
output_path = 'C:/Users/eholy/Desktop/SD/farfan_files/pool/'

# the max number of noise clips to be overlayed over the char clip
# for each char clip, a random number from 1 to max_overlays will be overlayed
max_overlays = 4

In [3]:
# create the output directories if they do not exist
if not os.path.exists(output_path):
    os.mkdir(output_path)
if not os.path.exists(output_path + 'clean'):
    os.mkdir(output_path + 'clean')
if not os.path.exists(output_path + 'noise'):
    os.mkdir(output_path + 'noise')
if not os.path.exists(output_path + 'mixture'):
    os.mkdir(output_path + 'mixture')

# open the .csv file and write the headers
header = ['', 'mixture_path', 'clean_path', 'noise_path']
f = open(output_path + 'metadata.csv', 'w', newline='')
writer = csv.writer(f)
writer.writerow(header)

37

In [4]:
# iterable object creation
char_clip_names = os.listdir(char_path)
noisepool_dirs = os.listdir(noisepool_path)

clip_count = 0
for char_clip_name in char_clip_names:
    #print(char_clip_name)
    
    char_clip, _ = librosa.load(char_path + char_clip_name)
    
    # randomly determine how many noise clips to overlay (inclusive, inclusive)
    to_overlay = random.randint(1, max_overlays)
    
    # iterate equal to to_overlay to build the select the noise clips
    noise_clips = []
    noisepool_dirs_used = []
    for i in range(0, to_overlay):
        # randomly select a noise character
        # cannot be a character that we have already used
        # because one person cannot speak over themselves in a conversation
        selected_dir = None
        while selected_dir == None or selected_dir in noisepool_dirs_used:
            selected_dir = noisepool_dirs[random.randrange(0, len(noisepool_dirs))]
        noisepool_dirs_used.append(selected_dir)
        
        # now we randomly select a clip from that noise character's dir
        selected_dir_clips = os.listdir(noisepool_path + selected_dir)
        selected_clip_name = selected_dir_clips[random.randrange(0, len(selected_dir_clips))]
        selected_clip, _ = librosa.load(noisepool_path + selected_dir + '/' + selected_clip_name)
        
        noise_clips.append(selected_clip)

    merged_noise_clip = np.array(sum(noise_clips))
    mixture_clip = np.array(sum([char_clip, merged_noise_clip]))
    
    # save the clean clip, the noise clip, and the overlay clips to file
    sf.write(output_path + 'clean/' + char_clip_name, char_clip, 22050)
    sf.write(output_path + 'noise/noiseclip' + str(clip_count) + '_' + str(to_overlay) + 'voices.wav', merged_noise_clip, 22050)
    sf.write(output_path + 'mixture/mixtureclip' + str(clip_count) + '.wav', mixture_clip, 22050)
    
    # mark the metadata in the .csv file
    metadata = [clip_count,
                'mixture/mixtureclip' + str(clip_count) + '.wav',
                'clean/' + char_clip_name,
                'noise/noiseclip' + str(clip_count) + '_' + str(to_overlay) + 'voices.wav'
               ]
    writer.writerow(metadata)
    clip_count += 1

# close the .csv file
f.close()
        
            

In [5]:
# # NORMALIZED AUDIO
# # COULD EXPLORE GETTING A RANDOM RANGE FOR DECIBEL LEVEL TO FIT INTO 
# # iterable object creation
# char_clip_names = os.listdir(char_path)
# noisepool_dirs = os.listdir(noisepool_path)

# clip_count = 0
# for char_clip_name in char_clip_names:
#     #print(char_clip_name)
    
#     #char_clip, _ = librosa.load(char_path + char_clip_name)
#     char_clip = AudioSegment.from_file(char_path + char_clip_name, format="wav")
#     char_dbfs = char_clip.dBFS
#     char_clip.close()
#     # randomly determine how many noise clips to overlay (inclusive, inclusive)
#     to_overlay = random.randint(1, max_overlays)
    
#     # iterate equal to to_overlay to build the select the noise clips
#     noise_clips = []
#     noisepool_dirs_used = []
#     for i in range(0, to_overlay):
#         # randomly select a noise character
#         # cannot be a character that we have already used
#         # because one person cannot speak over themselves in a conversation
#         selected_dir = None
#         while selected_dir == None or selected_dir in noisepool_dirs_used:
#             selected_dir = noisepool_dirs[random.randrange(0, len(noisepool_dirs))]
#         noisepool_dirs_used.append(selected_dir)
        
#         # now we randomly select a clip from that noise character's dir
#         selected_dir_clips = os.listdir(noisepool_path + selected_dir)
#         selected_clip_name = selected_dir_clips[random.randrange(0, len(selected_dir_clips))]
#         #selected_clip, _ = librosa.load(noisepool_path + selected_dir + '/' + selected_clip_name)
#         selected_clip = AudioSegment.from_file(noisepool_path + selected_dir + '/' + selected_clip_name, format="wav")
#         selected_clip = selected_clip.apply_gain(char_dbfs - selected_clip.dBFS)
#         print(selected_clip.get_array_of_samples())
#         noise_clips.append(selected_clip.get_array_of_samples())

#     merged_noise_clip = np.array(sum(noise_clips))
#     mixture_clip = np.array(sum([char_clip.get_array_of_samples(), merged_noise_clip]))
    
#     # save the clean clip, the noise clip, and the overlay clips to file
#     sf.write(output_path + 'clean/' + char_clip_name, char_clip, 22050)
#     sf.write(output_path + 'noise/noiseclip' + str(clip_count) + '_' + str(to_overlay) + 'voices.wav', merged_noise_clip, 22050)
#     sf.write(output_path + 'mixture/mixtureclip' + str(clip_count) + '.wav', mixture_clip, 22050)
    
#     # mark the metadata in the .csv file
#     metadata = [clip_count,
#                 'mixture/mixtureclip' + str(clip_count) + '.wav',
#                 'clean/' + char_clip_name,
#                 'noise/noiseclip' + str(clip_count) + '_' + str(to_overlay) + 'voices.wav'
#                ]
#     writer.writerow(metadata)
#     clip_count += 1

# # close the .csv file
#f.close()